# DEMO TOOL TRADITIONAL IRISH MUSIC CLASSIFIER FOR SCORES

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elloza/DIGIFOLK-USAL-ITMA/blob/master/researchline1/tool/DemoTool.ipynb)


This is the demo tool for deploying the traditional Irish music classifier for scores.

<div align="center">
  <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
    <img src="https://usal.es/files/logo_usal.png" alt="Logo" width="250" height="100" style="margin:10px;padding:20px;">
  </a>
  <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
    <img src="https://www.itma.ie/wp-content/themes/ITMA/images/itma-logo.svg" alt="Logo" width="250" height="100" style="margin:10px;padding:20px;">
  </a>
</div>

## Install all the required libraries


In [ ]:
# Install pycaret
!pip install pycaret[full]
# install gradio
!pip install -U gradio

# MUSIC FILE CONVERTER CLASSES

In [ ]:
%pip install midi2audio
!sudo apt-get update && sudo apt-get install fluidsynth -y && sudo apt-get install abcmidi -y
%pip install git+https://github.com/SpotlightKid/abc2xml

In [ ]:
import os
import midi2audio
from midi2audio import FluidSynth
import subprocess


class MusicFileConverter:
    def __init__(self, sound_font="sound_font.sf2", sample_rate=44100, abc2xml_path_file="abc2xml.py", xml2abc_path_file="xml2abc.py"):
        self._supported_formats = {
            "abc": [".abc", ".txt"],
            "midi": [".mid", ".midi"],
            "xml": [".xml"],
            "wav": [".wav"],
        }
        self.sound_font = sound_font
        self.sample_rate = sample_rate
        self.abc2xml_path_file = abc2xml_path_file
        self.xml2abc_path_file = xml2abc_path_file

        # Initialize FluidSynth
        FluidSynth(self.sound_font)
        FluidSynth(sample_rate=self.sample_rate)

    def _check_format(self, filename, file_format):
        _, ext = os.path.splitext(filename)
        return ext.lower() in self._supported_formats.get(file_format, [])

    def midi_to_wav(self, midi_file, wav_file):
        if not self._check_format(midi_file, "midi") or not self._check_format(
            wav_file, "wav"
        ):
            print("Invalid file format!")
            return None

        FluidSynth().midi_to_audio(midi_file, wav_file)

        return wav_file

    def midi_to_abc(self, midi_file, abc_file):
        if not self._check_format(midi_file, "midi") or not self._check_format(
            abc_file, "abc"
        ):
            print("Invalid file format!")
            return None

        # Use midi2abc to convert midi to abc
        subprocess.run(["midi2abc", midi_file, "-o", abc_file])

        return abc_file

    def abc_to_midi(self, abc_file, midi_file):
        if not self._check_format(abc_file, "abc") or not self._check_format(
            midi_file, "midi"
        ):
            print("Invalid file format!")
            return None

        # Use abcmidi to convert abc to midi
        subprocess.run(["abc2midi", abc_file, "-o", midi_file])

        return midi_file

    def abc_to_xml(self, abc_file, xml_folder):
        if not self._check_format(abc_file, "abc"):
            print("Invalid file format!")
            return None
        
        #  Make subprocess call to abc2xml.py 
        subprocess.run(["python", self.abc2xml_path_file, "-o", xml_folder, abc_file])

        return abc_file.replace(".abc", ".xml")

    def xml_to_abc(self, xml_file, abc_folder):
        if not self._check_format(xml_file, "xml"):
            print("Invalid file format!")
            return None

        # Implement the conversion from xml to abc using the external library
        subprocess.run(["python", self.xml2abc_path_file, "-o", abc_folder, xml_file])

        # Return the path of the abc file
        path_abc = self.get_parent_folder(xml_file) + "/" + self.get_file_name(xml_file).replace(".xml", ".abc")

        return path_abc

    def xml_to_midi(self, xml_file, midi_file):
        if not self._check_format(xml_file, "xml") or not self._check_format(
            midi_file, "midi"
        ):
            print("Invalid file format!")
            return

        # Implement the conversion from xml to midi using the external library
        # First convert xml to abc
        abc_file = self.xml_to_abc(xml_file, self.get_parent_folder(xml_file))
        # Then convert abc to midi
        midi_file = self.abc_to_midi(abc_file, midi_file)

        # Delete abc file
        os.remove(abc_file)

        return midi_file


    def xml_to_wav(self, xml_file, wav_file):
        if not self._check_format(xml_file, "xml") or not self._check_format(
            wav_file, "wav"
        ):
            print("Invalid file format!")
            return

        # Implement the conversion from xml to wav using the external library
        # First convert xml to midi
        midi_file = self.xml_to_midi(xml_file, self.get_file_name(xml_file).replace(".xml", ".mid"))

        # Then convert midi to wav
        wav_file = self.midi_to_wav(midi_file, wav_file)

        # Delete midi file
        os.remove(midi_file)

        return wav_file


    def wav_to_midi(self, wav_file, midi_file):
        if not self._check_format(wav_file, "wav") or not self._check_format(
            midi_file, "midi"
        ):
            print("Invalid file format!")
            return

        # Implement the conversion from wav to midi using the external library

        # Launch exception not implemented yet
        raise NotImplementedError("THIS REQUIRES SAGESHEET ENDPOINT- Coming soon!")

    # function which extrat the name of the file from the path
    def get_file_name(self, file_path):
        return os.path.basename(file_path)
    
    # function which extract the path of the parent folder of the file path
    def get_parent_folder(self, file_path):
        return os.path.dirname(file_path)

# Execute Gradio from here

In [ ]:
!pwd

In [ ]:
import pycaret
from pycaret.classification import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# GENRES
N_GENRES = 15

classifier = load_model("embedding_mule_best_model")
# TODO: CHANGE PATHS!
converter = MusicFileConverter(abc2xml_path_file="/home/jovyan/abc2xml.py", xml2abc_path_file="/home/jovyan/xml2abc.py")

def save_uploaded_file(uploaded_file):
    # Obtener el nombre del archivo y definir una ruta de guardado
    file_name = uploaded_file.name
    save_path = os.path.join("./uploads", file_name)
    
    # Asegurarse de que el directorio "uploads" exista
    os.makedirs("./uploads", exist_ok=True)
    
    # Guardar el archivo
    with open(save_path, 'wb') as f:
        f.write(uploaded_file.read())
    
    return save_path

def transform_file_to_wav(file):
    pass

def transform_wav_to_embedding_row(wav_file):
    pass

def classify_file(uploaded_file):
    # Supongamos que nuestro modelo nos da scores para tres clases: A, B y C
    # Estos son valores de ejemplo
    global classifier
    global converter

    file_path = save_uploaded_file(uploaded_file)

    # 1 - Transform abc, mid, or xml file to wav file
    # Check if file is abc, mid, or xml
    if file_path.endswith(".abc"):
        wav_file = converter.convert_abc_to_wav()
    elif file_path.endswith(".mid"):
        wav_file = converter.convert_mid_to_wav()
    elif file_path.endswith(".xml"):
        wav_file = converter.convert_xml_to_wav()
    else:
        raise Exception("File format not supported")

    wav_file = transform_file_to_wav(file)

    # 2 - Transform wav file to embedding row with columns embedding_mule_0, embedding_mule__1, etc
    row = transform_wav_to_embedding_row(wav_file)

    # 3 - Perform the inference
    pred = predict_model(classifier, row, raw_score=True)

    # Get only the columns called prediction_score_*
    pred = pred.filter(regex="prediction_score_*")

    # Get pred as a dictionary where keys are the columns names and values are the columns values
    pred_dict = pred.to_dict("records")[0]

    # Rename the keys of the dictionary to remove the prefix prediction_score_
    pred_dict = {key.replace("prediction_score_", ""): value for key, value in pred_dict.items()}

    return pred_dict


import gradio as gr

# Logo y título personalizado utilizando HTML
title_with_logo = """
    <div style='text-align: center;'>
        <div style='margin-bottom: 10px; font-weight: bold;'>
            IRISH TRADITIONAL MUSIC GENRE CLASSIFICATION
        </div>
        <div style='margin-bottom: 10px; font-weight: regular;'>
            Identifying Irish traditional music genres using latent audio representations
        </div>
        <div style='display: flex; justify-content: space-between; align-items: center;'>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://usal.es/files/logo_usal.png" alt="Logo" width="150" height="50" style="margin:10px;padding:20px;">
            </a>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://www.itma.ie/wp-content/themes/ITMA/images/itma-logo.svg" alt="Logo" width="150" height="50" style="margin:10px;padding:20px;">
            </a>
            <a href="https://github.com/elloza/DIGIFOLK-USAL-ITMA">
                <img src="https://cordis.europa.eu/images/logo/logo-ec-es.svg" alt="Logo" width="150" height="50" style="margin:10px;padding:20px;">
            </a>
        </div>
    </div>
"""

with gr.Blocks(title="ITMA Classifier",theme=gr.themes.Default(primary_hue="emerald")) as demo:
    iface = gr.Interface(
        fn=classify_file,
        inputs=gr.inputs.File(label="Upload .abc, .mid, or .xml file"),
        outputs=gr.outputs.Label(num_top_classes=N_GENRES),
        live=False,
        title=title_with_logo,
        allow_flagging="never"
    )

# Para compartir el enlace
# iface.share()

# Si solo quieres lanzarlo localmente, usa
demo.launch(inbrowser=True,debug=True)

## Execute the application

## Deploy Gradio Application